In [1]:
import pandas as pd
import datetime as dt

from tqdm import tqdm_notebook

In [9]:
'''
Import data sources:
1. Additional aggreements
2. Supply schedules
3. NCG daily prices
4. UEB monthly prices
'''

data_addcontr = pd.read_excel('04_output/addcontr_all_clean_v1.5.xlsx', 'contracts_short')
data_schedule = pd.read_excel('04_output/addcontr_all_clean_v1.5.xlsx', 'schedule')

data_addcontr.loc[:,'addcontr_date'] = pd.to_datetime(data_addcontr['addcontr_date']).map(lambda x: x.date())
data_addcontr = data_addcontr.loc[~data_addcontr['price_addcontr'].isin(['not a contract'])]

NCG_prices = pd.read_csv('02_data/PNCGMc1_2019-01-14.csv')
NCG_prices.loc[:,'Date'] = pd.to_datetime(NCG_prices['Date']).map(lambda x: x.date())

ueb_prices = pd.read_csv('02_data/prices_ueb_2019_01.csv')

print(data_schedule.shape)
data_addcontr.head(3)

(8392, 5)


,addcontr_date,rationale_type,rationale,contract_id,date_signed,contract_end,tender_id,procuring_entity,contract_value,region,tenderURL,price_start,price_addcontr,row_added_manually
0,2018-06-12,itemPriceVariation,"п.2 ч.4 ст.36 ЗУ ""Про публічні закупівлі""",001de2081beb4e6cbe967805f79ac92f,2018-01-19,2018-12-31 00:00:00,UA-2017-12-07-000132-b,КЗ Хмельницький обласний центр фізичного вихов...,1702320.0,Хмельницька область,https://prozorro.gov.ua/tender/UA-2017-12-07-0...,10380.0,10800.0,True
1,2018-08-27,itemPriceVariation,"п.2 ч.4 ст.36 ЗУ ""Про публічні закупівлі""",001de2081beb4e6cbe967805f79ac92f,2018-01-19,2018-12-31 00:00:00,UA-2017-12-07-000132-b,КЗ Хмельницький обласний центр фізичного вихов...,1702320.0,Хмельницька область,https://prozorro.gov.ua/tender/UA-2017-12-07-0...,10380.0,10980.0,True
2,2018-10-04,itemPriceVariation,"п.2 ч.4 ст.36 ЗУ ""Про публічні закупівлі""",001de2081beb4e6cbe967805f79ac92f,2018-01-19,2018-12-31 00:00:00,UA-2017-12-07-000132-b,КЗ Хмельницький обласний центр фізичного вихов...,1702320.0,Хмельницька область,https://prozorro.gov.ua/tender/UA-2017-12-07-0...,10380.0,11490.0,False


In [10]:
'''
Functions to generate data for the final dataframe with potential economy
'''

contract_ids = data_addcontr['contract_id'].unique()

'''
Connect additional agreement dates to the supply schedules
Additional agreement is connected, if it was signed no later than 15th day of the respective month
The latest relevant additional agreement date is taken
If no additional aggrement had been signed yet, the initial contract date is taken
'''
def imply_addcontr_date(month,contract_id):
    data_addcontr_cut = data_addcontr.loc[data_addcontr['contract_id']==contract_id].reset_index()
    date_contr_signed = data_addcontr_cut['date_signed'][0].date()

    contracts_after = [datecontr for datecontr in data_addcontr_cut['addcontr_date'] 
                       if (datecontr - month <= dt.timedelta(15)) and (datecontr - month >= dt.timedelta(0))]
    
    contracts_before = [datecontr for datecontr in data_addcontr_cut['addcontr_date']
                       if (datecontr - month <= dt.timedelta(0))]

    if len(contracts_after) > 0:
        date_addcontr = max(contracts_after)
    elif len(contracts_before) > 0:
        date_addcontr = max(contracts_before)
    else:
        date_addcontr = date_contr_signed

    return date_addcontr

'''
Get the price, which was active during each month of the supply,
based on the date, when the additional agreement was signed (see imply_addcontr_date()).
If several agreement were signed in one day, the highest price is taken
'''
def imply_current_price(addcontr_date,contract_id):
    data_addcontr_cut = data_addcontr.loc[data_addcontr['contract_id'] == contract_id]
    addcontr_row = data_addcontr_cut.loc[data_addcontr_cut['addcontr_date'] == addcontr_date]
    if len(addcontr_row)==1:
        current_price = addcontr_row.iloc[0,:]['price_addcontr']
    elif len(addcontr_row)==0:
        current_price = data_addcontr_cut.iloc[0,:]['price_start']
    elif len(addcontr_row['addcontr_date'].unique())==1:
        current_price = max(addcontr_row['price_addcontr'])
    else:
        print(len(addcontr_row['addcontr_date'].unique()))
        raise ValueError('Selection criteria are not unique')
    return current_price

'''
Get mean gas price on NCG for the last 20 days before the additional agreement was signed
'''
def ncg_mean(date_index):
    date_signed = date_index - dt.timedelta(20)
    return NCG_prices.loc[(NCG_prices['Date'] >= date_signed) & (NCG_prices['Date'] <= date_index)]['Last'].mean()

'''
Get mean exchange rate for hryvnias to euros for the last 5 days before the additional agreement was signed
'''
def exchange_rate_mean(date_index):
    date_signed = date_index - dt.timedelta(5)
    return exchange_rates.loc[(exchange_rates['date'] >= date_signed) & (exchange_rates['date'] <= date_index)]['exchange_rate_eur'].mean()

'''
Get relevant monthly UEB natural gas price
'''
def ueb_price(date_input):
    ueb_price = ueb_prices.loc[ueb_prices['month'] == date_input.month]
    ueb_price = ueb_price.loc[ueb_price['year'] == date_input.year]['price'].reset_index()
    ueb_price = ueb_price['price'][0]
    return ueb_price

'''
Get initial contract price for each contract
'''
def get_init_price(contract_id):
    data_loc = data_addcontr.loc[data_addcontr['contract_id']==contract_id].reset_index()
    return data_loc['price_start'][0]

'''
Get the date, when initial contract had been signed
'''
def get_init_date(contract_id):
    data_loc = data_addcontr.loc[data_addcontr['contract_id']==contract_id].reset_index()
    return data_loc['date_signed'][0].date()

In [11]:
'''
TEST PLACE
Connect relevant additional agreement dates and prices to the supply schedule for a single contract id
'''
no_schedules = []
data_schedule_cut = data_schedule[['contract_id','month','year','amount_1000m3','supply_implied']]
data_schedule_cut = data_schedule_cut.loc[data_schedule_cut['contract_id']==contract_ids[44]]

if len(data_schedule_cut) > 0:
    data_schedule_cut['start_date'] = data_schedule_cut.apply(lambda x: dt.date(x['year'],x['month'],1), axis = 1)
    data_schedule_cut['addcontr_date'] = data_schedule_cut.apply(lambda x: imply_addcontr_date(x['start_date'],x['contract_id']), axis = 1)
    data_schedule_cut['price'] = data_schedule_cut.apply(lambda x: imply_current_price(x['addcontr_date'],x['contract_id']), axis = 1)
elif len(data_schedule_cut) == 0:
    print('Для contract_id %s відсутній графік поставок' % contract_ids[44])
    no_schedules.append(contract_ids[44])
data_schedule_cut

,contract_id,month,year,amount_1000m3,supply_implied,start_date,addcontr_date,price
6627,0a61282b67fc4c93b84b862b851562a8,2,2018,189.065,False,2018-02-01,2018-02-07,9960.00
6628,0a61282b67fc4c93b84b862b851562a8,3,2018,187.555,False,2018-03-01,2018-03-13,9960.00
6629,0a61282b67fc4c93b84b862b851562a8,4,2018,29.424,False,2018-04-01,2018-03-13,9960.00
6630,0a61282b67fc4c93b84b862b851562a8,5,2018,1.972,False,2018-05-01,2018-03-13,9960.00
6631,0a61282b67fc4c93b84b862b851562a8,6,2018,0.950,False,2018-06-01,2018-03-13,9960.00
6632,0a61282b67fc4c93b84b862b851562a8,7,2018,1.000,False,2018-07-01,2018-06-20,10794.00
6633,0a61282b67fc4c93b84b862b851562a8,8,2018,1.000,False,2018-08-01,2018-06-20,10794.00
6634,0a61282b67fc4c93b84b862b851562a8,9,2018,1.000,False,2018-09-01,2018-09-09,12625.80
6635,0a61282b67fc4c93b84b862b851562a8,10,2018,60.000,False,2018-10-01,2018-10-03,13762.08
6636,0a61282b67fc4c93b84b862b851562a8,11,2018,122.400,False,2018-11-01,2018-10-03,13762.08


In [12]:
'''
Connect relevant additional agreement dates and prices to the supply schedule for all contracts,
as well as baseline prices, de facto price change and expenses.

Some contracts do not have schedules, the usual reason is zero supply (contract terminated),
but might be some errors.
All such cases are recorded and checked manually
'''

no_schedules = []
data_schedule_new = pd.DataFrame()

for contract_id in tqdm_notebook(contract_ids):
    data_schedule_loc = data_schedule[['contract_id','month','year','amount_1000m3','supply_implied']]
    data_schedule_loc = data_schedule_loc.loc[data_schedule_loc['contract_id']==contract_id]
    
    if len(data_schedule_loc) > 0:
        data_schedule_loc['start_date'] = data_schedule_loc.apply(lambda x: dt.date(x['year'],x['month'],1), axis = 1)
        data_schedule_loc['addcontr_date'] = data_schedule_loc.apply(lambda x: imply_addcontr_date(x['start_date'],x['contract_id']), axis = 1)
        data_schedule_loc['price'] = data_schedule_loc.apply(lambda x: imply_current_price(x['addcontr_date'],x['contract_id']), axis = 1)
        data_schedule_loc = data_schedule_loc.reset_index()

        baseline_dates = data_schedule_loc['addcontr_date'].shift()
        data_schedule_loc['baseline_date'] = baseline_dates.fillna(get_init_date(contract_id))

        baseline_prices = data_schedule_loc['price'].shift()
        data_schedule_loc['baseline_price'] = baseline_prices.fillna(get_init_price(contract_id))
        
        if len(data_schedule_new) == 0:
            data_schedule_new = data_schedule_loc
        else:
            data_schedule_new = data_schedule_new.append(data_schedule_loc, ignore_index = True)
    elif len(data_schedule_loc) == 0:
        print('Для contract_id %s відсутній графік поставок' % contract_id)
        no_schedules.append(contract_id)
           
data_schedule_new = data_schedule_new.drop(columns = ['index'])
data_schedule_new.loc[:,'price_change_noformula'] = data_schedule_new.loc[:,'price'] - data_schedule_new.loc[:,'baseline_price']
data_schedule_new.loc[:,'expenses_fact'] = data_schedule_new.loc[:,'price']*data_schedule_new.loc[:,'amount_1000m3']
    
df_no_schedules = pd.DataFrame({'contract_id':no_schedules})
df_no_schedules.to_csv('03_for_students/03_purgatorium/no_schedules_2019-14-01.csv', index = False)

print(data_schedule_new.shape)
print(no_schedules)
data_schedule_new.head()

Для contract_id 09583d8ea28d4c3b898e45382eacf5ae відсутній графік поставок
Для contract_id 25f40e0f8739410ca896dca59eed3d3f відсутній графік поставок
Для contract_id 2c314dbf12324b94b38a77f23e4e6731 відсутній графік поставок
Для contract_id 6869590a35a84749b6fd50f1f494b819 відсутній графік поставок
Для contract_id 7400394b12bb489d89ad3b6735441225 відсутній графік поставок
Для contract_id 978db960fd92485ca2b2e150cedcd333 відсутній графік поставок
Для contract_id aba3178a8adc4f3da8b25a41ce2f016e відсутній графік поставок
Для contract_id ff67ba7bab5d4d1ea6280aa59b2b4df3 відсутній графік поставок

(8384, 12)
['09583d8ea28d4c3b898e45382eacf5ae', '25f40e0f8739410ca896dca59eed3d3f', '2c314dbf12324b94b38a77f23e4e6731', '6869590a35a84749b6fd50f1f494b819', '7400394b12bb489d89ad3b6735441225', '978db960fd92485ca2b2e150cedcd333', 'aba3178a8adc4f3da8b25a41ce2f016e', 'ff67ba7bab5d4d1ea6280aa59b2b4df3']


,contract_id,month,year,amount_1000m3,supply_implied,start_date,addcontr_date,price,baseline_date,baseline_price,price_change_noformula,expenses_fact
0,001de2081beb4e6cbe967805f79ac92f,1,2018,30.0,False,2018-01-01,2018-01-19,10380.0,2018-01-19,10380.0,0.0,311400.0
1,001de2081beb4e6cbe967805f79ac92f,2,2018,26.0,False,2018-02-01,2018-01-19,10380.0,2018-01-19,10380.0,0.0,269880.0
2,001de2081beb4e6cbe967805f79ac92f,3,2018,20.0,False,2018-03-01,2018-01-19,10380.0,2018-01-19,10380.0,0.0,207600.0
3,001de2081beb4e6cbe967805f79ac92f,4,2018,9.0,False,2018-04-01,2018-01-19,10380.0,2018-01-19,10380.0,0.0,93420.0
4,001de2081beb4e6cbe967805f79ac92f,5,2018,7.0,False,2018-05-01,2018-01-19,10380.0,2018-01-19,10380.0,0.0,72660.0


In [13]:
'''
Import exchange rates data

Append data, which will be used for the indexation calculations later:
1. Exchange rate (baseline and mean)
2. NCG price (baseline and mean)
3. UEB price (baseline and mean)
'''

exchange_rates = pd.read_csv('02_data/exchange_rates_eur_2018_12_30.csv')
exchange_rates.loc[:,'date'] = pd.to_datetime(exchange_rates.loc[:,'date']).map(lambda x: x.date())
exchange_rates.loc[:,'exchange_rate_eur'] = exchange_rates.loc[:,'exchange_rate_eur'].map(lambda x: x/100)

data_schedule_all = data_schedule_new.merge(exchange_rates, left_on = 'baseline_date', right_on = 'date', how = 'left')
data_schedule_all = data_schedule_all.merge(NCG_prices, left_on = 'baseline_date', right_on = 'Date', how = 'left')
data_schedule_all = data_schedule_all.drop(columns = ['date','Date'])
data_schedule_all = data_schedule_all.rename(columns = {'exchange_rate_eur':'exchange_rate_base','Last':'ncg_base'})

data_schedule_all['ncg_mean'] = data_schedule_all['addcontr_date'].map(lambda x: ncg_mean(x))
data_schedule_all['exchange_rate_index'] = data_schedule_all['addcontr_date'].map(lambda x: exchange_rate_mean(x))

data_schedule_all['ueb_baseline'] = data_schedule_all['baseline_date'].map(lambda x: ueb_price(x))
data_schedule_all['ueb_index'] = data_schedule_all['addcontr_date'].map(lambda x: ueb_price(x))

print(data_schedule_all.shape)
data_schedule_all.head()

(8384, 18)


,contract_id,month,year,amount_1000m3,supply_implied,start_date,addcontr_date,price,baseline_date,baseline_price,price_change_noformula,expenses_fact,exchange_rate_base,ncg_base,ncg_mean,exchange_rate_index,ueb_baseline,ueb_index
0,001de2081beb4e6cbe967805f79ac92f,1,2018,30.0,False,2018-01-01,2018-01-19,10380.0,2018-01-19,10380.0,0.0,311400.0,35.209253,18.091,19.156619,34.85388,9683.94,9683.94
1,001de2081beb4e6cbe967805f79ac92f,2,2018,26.0,False,2018-02-01,2018-01-19,10380.0,2018-01-19,10380.0,0.0,269880.0,35.209253,18.091,19.156619,34.85388,9683.94,9683.94
2,001de2081beb4e6cbe967805f79ac92f,3,2018,20.0,False,2018-03-01,2018-01-19,10380.0,2018-01-19,10380.0,0.0,207600.0,35.209253,18.091,19.156619,34.85388,9683.94,9683.94
3,001de2081beb4e6cbe967805f79ac92f,4,2018,9.0,False,2018-04-01,2018-01-19,10380.0,2018-01-19,10380.0,0.0,93420.0,35.209253,18.091,19.156619,34.85388,9683.94,9683.94
4,001de2081beb4e6cbe967805f79ac92f,5,2018,7.0,False,2018-05-01,2018-01-19,10380.0,2018-01-19,10380.0,0.0,72660.0,35.209253,18.091,19.156619,34.85388,9683.94,9683.94


In [15]:
'''
Apply indexation formula to each row (which corresponds to a month of supply according to a single contract)
'''

#baseline_price*exchange_rate_index/exchange_rate_base + (ncg_mean-ncg_base)*10.57*exchange_rate_index*1.2*0.95 + (ueb_index-ueb_baseline)*0.05
def count_price_formula(x):
    if x['addcontr_date'] == x['baseline_date']:
        return x['baseline_price']
    else:
        return x['baseline_price']*x['exchange_rate_index']/x['exchange_rate_base'] + \
            (x['ncg_mean']-x['ncg_base'])*10.57*x['exchange_rate_index']*1.2*0.95 + \
            (x['ueb_index']-x['ueb_baseline'])*0.05

data_schedule_all['price_formula'] = data_schedule_all.apply(lambda x: count_price_formula(x), axis = 1)

data_schedule_all.loc[:,'price_change_formula'] = data_schedule_all.loc[:,'price_formula'] - data_schedule_all.loc[:,'baseline_price']
data_schedule_all.loc[:,'expenses_formula'] = data_schedule_all.loc[:,'price_formula']*data_schedule_all.loc[:,'amount_1000m3']
data_schedule_all.loc[:,'potential_economy'] = data_schedule_all.loc[:,'expenses_fact'] - data_schedule_all.loc[:,'expenses_formula']

data_schedule_all.head()

,contract_id,month,year,amount_1000m3,supply_implied,start_date,addcontr_date,price,baseline_date,baseline_price,...,exchange_rate_base,ncg_base,ncg_mean,exchange_rate_index,ueb_baseline,ueb_index,price_formula,price_change_formula,expenses_formula,potential_economy
0,001de2081beb4e6cbe967805f79ac92f,1,2018,30.0,False,2018-01-01,2018-01-19,10380.0,2018-01-19,10380.0,...,35.209253,18.091,19.156619,34.85388,9683.94,9683.94,10380.0,0.0,311400.0,0.0
1,001de2081beb4e6cbe967805f79ac92f,2,2018,26.0,False,2018-02-01,2018-01-19,10380.0,2018-01-19,10380.0,...,35.209253,18.091,19.156619,34.85388,9683.94,9683.94,10380.0,0.0,269880.0,0.0
2,001de2081beb4e6cbe967805f79ac92f,3,2018,20.0,False,2018-03-01,2018-01-19,10380.0,2018-01-19,10380.0,...,35.209253,18.091,19.156619,34.85388,9683.94,9683.94,10380.0,0.0,207600.0,0.0
3,001de2081beb4e6cbe967805f79ac92f,4,2018,9.0,False,2018-04-01,2018-01-19,10380.0,2018-01-19,10380.0,...,35.209253,18.091,19.156619,34.85388,9683.94,9683.94,10380.0,0.0,93420.0,0.0
4,001de2081beb4e6cbe967805f79ac92f,5,2018,7.0,False,2018-05-01,2018-01-19,10380.0,2018-01-19,10380.0,...,35.209253,18.091,19.156619,34.85388,9683.94,9683.94,10380.0,0.0,72660.0,0.0


In [16]:
'''
Get the sum of potential economy
'''
print("Сума потенційної економії становить %.3f млн грн" %(data_schedule_all['potential_economy'].sum()/10**6))
print("Опрацьовано %.3f %% усіх договорів на суму понад 1 млн грн" %(len(data_schedule_all['contract_id'].unique())/1225*100))
print(len(data_schedule_all['contract_id'].unique()))

Сума потенційної економії становить 120.253 млн грн
Опрацьовано 98.122 % усіх договорів на суму понад 1 млн грн
1202


In [17]:
'''
Export dataframe with all calculations to xlsx
'''
data_schedule_all.to_excel('04_output/potential_economy_v2.1.xlsx', index = False)